In [181]:
# Import
import requests
import pandas as pd
import numpy as np

import re

import urllib
import bs4
from urllib import request

In [182]:
# Adds the part about the user right here

In [183]:
# Data Extraction

# Query to extract a specific frame of time. We have to change it for the user desired date
query = "date_start < 2022-12-19 AND date_start > 2021-12-12"
url = "https://data.iledefrance.fr/api/records/1.0/search/?dataset=evenements-publics-cibul&q=&rows=100&q=(" + query + ")&sort=date_start"
req = requests.get(url)

df = pd.json_normalize(req.json()['records'])

important_keys = ["fields.title", "fields.latlon", "fields.date_start", "fields.tags", 'fields.title', "fields.pricing_info", "fields.description",'fields.link', 'fields.free_text', 'fields.address', 'fields.department','fields.placename', 'fields.region', 'fields.date_end', 'fields.lang', 'geometry.type', 'geometry.coordinates', 'fields.city_district']
df = df[important_keys]

dict_important_keys = {elt : elt.replace("fields.", "") for elt in important_keys}
df.rename(columns = dict_important_keys, inplace=True)

df_to_treat = df.copy()
df_to_treat

,title,latlon,date_start,tags,title,pricing_info,description,link,free_text,address,department,placename,region,date_end,lang,geometry.type,geometry.coordinates,city_district
0,Colloque AVISA,"[48.631891, 2.431379]",2022-12-09,NaN,Colloque AVISA,Inscription,Écrire l’histoire du harcèlement sexuel sur la...,http://openagenda.com/event/colloque-avisa,Colloque en hybride,2 rue Facteur Cheval - Evry-Courcouronnes,Essonne,Université d'Évry - UFR SHS,Île-de-France,2022-12-10,fr,Point,"[2.431379, 48.631891]",Évry
1,Le Carnaval des Animaux Sud-américains,"[48.889308, 2.393804]",2022-11-20,"Ensemble ALMAVIVA,Carnaval des animaux sud-amé...",Le Carnaval des Animaux Sud-américains,NaN,"Bienvenue à Rio de Janeiro, qui accueille le c...",http://openagenda.com/event/le-carnaval-des-an...,Doté depuis son enfance à Rio du pouvoir de co...,"221 avenue Jean-Jaurès, 75019 Paris",Paris,La Philharmonie de Paris,Île-de-France,2022-11-20,fr,Point,"[2.393804, 48.889308]",NaN
2,Logique des passions (titre provisoire),"[48.800061, 2.403721]",2022-11-17,"théâtre,immersion,installation,numérique,vidéo...",Logique des passions (titre provisoire),"sur réservations , tarifs 13 euros TP et 10 eu...",Le Diptyque Collectif,http://openagenda.com/event/logique-des-passio...,Ce spectacle invite le public à vivre ou reviv...,"13, rue Pierre Sémard. 94400 Vitry-sur-Seine",Val-de-Marne,Nouveau Gare au Théâtre,Île-de-France,2022-11-19,fr,Point,"[2.403721, 48.800061]",Port à l'Anglais
3,Expoprotection,"[48.832528, 2.287004]",2022-11-15,"expoprotection,santé et sécurité au travail,sé...",Expoprotection,Demandez votre badge gratuit,Le salon de la prévention et de la gestion des...,http://openagenda.com/event/expoprotection_640,"Tous les deux ans à Paris, Expoprotection rass...","Place de la Porte de Versailles, 75015 Paris",Paris,Porte de Versailles,Île-de-France,2022-11-17,fr,Point,"[2.287004, 48.832528]",NaN
4,EquipHotel,"[48.832488, 2.287499]",2022-11-06,"hôtellerie,restauration,collectivités",EquipHotel,NaN,Hotel &amp; Restaurant Business Place,http://openagenda.com/event/equiphotel_359,"Du 6 au 10 novembre 2022, ne manquez pas le re...",1 place de la Porte de Versailles,Paris,Paris expo Porte de Versailles,Île-de-France,2022-11-10,fr,Point,"[2.287499, 48.832488]",15e Arrondissement
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Sirba Octet &amp; Balalaïka,"[48.765358, 2.268554]",2022-06-07,NaN,Sirba Octet &amp; Balalaïka,Tarif plein : 20€ / réduit : 15€ / super rédui...,SIRBALALAÏKA,http://openagenda.com/event/test-sirba-octet-a...,"Voir un concert du Sirba Octet, c’est comme pa...","254, avenue de la Division Leclerc 92290 châte...",Hauts-de-Seine,L'Azimut - Théâtre La Piscine,Île-de-France,2022-06-07,fr,Point,"[2.268554, 48.765358]",NaN
96,L'Aparté vous accueille ce samedi,"[48.824529, 2.274148]",2022-06-04,"CLAVIM,Aparté,Maison Verte",L'Aparté vous accueille ce samedi,Entrée libre sans rendez-vous,Lieu d’accueil pour les bébés et les enfants d...,http://openagenda.com/event/laparte-vous-accue...,L’**Aparté** vous accueille sans rendez-vous c...,"60 rue du Général Leclerc, 92130 Issy-les-Moul...",Hauts-de-Seine,L'Aparté,Île-de-France,2022-06-04,fr,Point,"[2.274148, 48.824529]",NaN
97,Erstkommunion,"[48.870856, 2.285033]",2022-06-04,NaN,Erstkommunion,NaN,Festgottesdienst der Erstkommunion in St. Hono...,http://openagenda.com/event/erstkommunion-4029827,NaN,"66 bis avenue Raymond-Poincaré, 75116 Paris",Paris,St Honoré d'Eylau,Île-de-France,2022-06-04,fr,Point,"[2.285033, 48.870856]",Paris 16e Arrondissement
98,Solennité de Pentecôte,"[48.855518, 2.354704]",2022-06-04,NaN,Solennité de Pentecôte,NaN,Célébrer le don de l'Esprit Saint avec les Fra...,http://openagenda.com/event/solennite-de-pente...,* **Samedi 4 juin 2022**\n\n12h30 : messe du...,"13, rue des Barres, 75004 Paris",Paris,"Église Saint-Gervais,Paris",Île-de-France,2022-06-05,fr,Point,"[2.354704, 48.855518]",Paris 4e Arrondissement


In [187]:
def extract_price_regex(x):
    L = []
    for k in re.findall("\S+\s?€", str(x)):
        try:
            a = float(k.replace("€", "").replace(",", "."))
        except ValueError:
            # It doesn't work
            a = float("NaN")
        L.append(a)
    return np.mean(L)

df_to_treat['pricing_info'] = df['pricing_info'].apply(lambda x:extract_price_regex(x))

In [188]:
df_to_treat

,title,latlon,date_start,tags,title,pricing_info,description,link,free_text,address,department,placename,region,date_end,lang,geometry.type,geometry.coordinates,city_district
0,Colloque AVISA,"[48.631891, 2.431379]",2022-12-09,NaN,Colloque AVISA,NaN,Écrire l’histoire du harcèlement sexuel sur la...,http://openagenda.com/event/colloque-avisa,Colloque en hybride,2 rue Facteur Cheval - Evry-Courcouronnes,Essonne,Université d'Évry - UFR SHS,Île-de-France,2022-12-10,fr,Point,"[2.431379, 48.631891]",Évry
1,Le Carnaval des Animaux Sud-américains,"[48.889308, 2.393804]",2022-11-20,"Ensemble ALMAVIVA,Carnaval des animaux sud-amé...",Le Carnaval des Animaux Sud-américains,NaN,"Bienvenue à Rio de Janeiro, qui accueille le c...",http://openagenda.com/event/le-carnaval-des-an...,Doté depuis son enfance à Rio du pouvoir de co...,"221 avenue Jean-Jaurès, 75019 Paris",Paris,La Philharmonie de Paris,Île-de-France,2022-11-20,fr,Point,"[2.393804, 48.889308]",NaN
2,Logique des passions (titre provisoire),"[48.800061, 2.403721]",2022-11-17,"théâtre,immersion,installation,numérique,vidéo...",Logique des passions (titre provisoire),NaN,Le Diptyque Collectif,http://openagenda.com/event/logique-des-passio...,Ce spectacle invite le public à vivre ou reviv...,"13, rue Pierre Sémard. 94400 Vitry-sur-Seine",Val-de-Marne,Nouveau Gare au Théâtre,Île-de-France,2022-11-19,fr,Point,"[2.403721, 48.800061]",Port à l'Anglais
3,Expoprotection,"[48.832528, 2.287004]",2022-11-15,"expoprotection,santé et sécurité au travail,sé...",Expoprotection,NaN,Le salon de la prévention et de la gestion des...,http://openagenda.com/event/expoprotection_640,"Tous les deux ans à Paris, Expoprotection rass...","Place de la Porte de Versailles, 75015 Paris",Paris,Porte de Versailles,Île-de-France,2022-11-17,fr,Point,"[2.287004, 48.832528]",NaN
4,EquipHotel,"[48.832488, 2.287499]",2022-11-06,"hôtellerie,restauration,collectivités",EquipHotel,NaN,Hotel &amp; Restaurant Business Place,http://openagenda.com/event/equiphotel_359,"Du 6 au 10 novembre 2022, ne manquez pas le re...",1 place de la Porte de Versailles,Paris,Paris expo Porte de Versailles,Île-de-France,2022-11-10,fr,Point,"[2.287499, 48.832488]",15e Arrondissement
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Sirba Octet &amp; Balalaïka,"[48.765358, 2.268554]",2022-06-07,NaN,Sirba Octet &amp; Balalaïka,15.0,SIRBALALAÏKA,http://openagenda.com/event/test-sirba-octet-a...,"Voir un concert du Sirba Octet, c’est comme pa...","254, avenue de la Division Leclerc 92290 châte...",Hauts-de-Seine,L'Azimut - Théâtre La Piscine,Île-de-France,2022-06-07,fr,Point,"[2.268554, 48.765358]",NaN
96,L'Aparté vous accueille ce samedi,"[48.824529, 2.274148]",2022-06-04,"CLAVIM,Aparté,Maison Verte",L'Aparté vous accueille ce samedi,NaN,Lieu d’accueil pour les bébés et les enfants d...,http://openagenda.com/event/laparte-vous-accue...,L’**Aparté** vous accueille sans rendez-vous c...,"60 rue du Général Leclerc, 92130 Issy-les-Moul...",Hauts-de-Seine,L'Aparté,Île-de-France,2022-06-04,fr,Point,"[2.274148, 48.824529]",NaN
97,Erstkommunion,"[48.870856, 2.285033]",2022-06-04,NaN,Erstkommunion,NaN,Festgottesdienst der Erstkommunion in St. Hono...,http://openagenda.com/event/erstkommunion-4029827,NaN,"66 bis avenue Raymond-Poincaré, 75116 Paris",Paris,St Honoré d'Eylau,Île-de-France,2022-06-04,fr,Point,"[2.285033, 48.870856]",Paris 16e Arrondissement
98,Solennité de Pentecôte,"[48.855518, 2.354704]",2022-06-04,NaN,Solennité de Pentecôte,NaN,Célébrer le don de l'Esprit Saint avec les Fra...,http://openagenda.com/event/solennite-de-pente...,* **Samedi 4 juin 2022**\n\n12h30 : messe du...,"13, rue des Barres, 75004 Paris",Paris,"Église Saint-Gervais,Paris",Île-de-France,2022-06-05,fr,Point,"[2.354704, 48.855518]",Paris 4e Arrondissement


In [189]:
# Date : we extract the hours of opening

In [190]:
def get_hours_regex(text):
    return str(text.select('span.hour')[0]).replace('<span class="hour">', "").replace("</span>", "")
    
def get_text(link):
    request_text = request.urlopen(link).read()
    page = bs4.BeautifulSoup(request_text)
    return(page)
    
def get_timespan(df_link):
    L = []
    for link in df_link:
        try:
            text = get_text(link)
            hours = get_hours_regex(text)
            L.append(hours)
        except (urllib.error.HTTPError, IndexError): #403 error case, if the Event is not correctly setup and if the event is not accessible
            L.append(float("NaN"))
        
    return L

df_to_treat['opening_hours'] = get_timespan(df['link']) # Time consuming, so we should get all information at once to avoid doing too much webscraping

# Should we differency ponctual events (movies, plays, sports) and long running events (expositions) ?
# -> We can probably do it judging the timespan of the events in terms of days and hours, but is it useful ?

In [191]:
df_to_treat

,title,latlon,date_start,tags,title,pricing_info,description,link,free_text,address,department,placename,region,date_end,lang,geometry.type,geometry.coordinates,city_district,opening_hours
0,Colloque AVISA,"[48.631891, 2.431379]",2022-12-09,NaN,Colloque AVISA,NaN,Écrire l’histoire du harcèlement sexuel sur la...,http://openagenda.com/event/colloque-avisa,Colloque en hybride,2 rue Facteur Cheval - Evry-Courcouronnes,Essonne,Université d'Évry - UFR SHS,Île-de-France,2022-12-10,fr,Point,"[2.431379, 48.631891]",Évry,09:00 - 17:00
1,Le Carnaval des Animaux Sud-américains,"[48.889308, 2.393804]",2022-11-20,"Ensemble ALMAVIVA,Carnaval des animaux sud-amé...",Le Carnaval des Animaux Sud-américains,NaN,"Bienvenue à Rio de Janeiro, qui accueille le c...",http://openagenda.com/event/le-carnaval-des-an...,Doté depuis son enfance à Rio du pouvoir de co...,"221 avenue Jean-Jaurès, 75019 Paris",Paris,La Philharmonie de Paris,Île-de-France,2022-11-20,fr,Point,"[2.393804, 48.889308]",NaN,11:00 - 12:00
2,Logique des passions (titre provisoire),"[48.800061, 2.403721]",2022-11-17,"théâtre,immersion,installation,numérique,vidéo...",Logique des passions (titre provisoire),NaN,Le Diptyque Collectif,http://openagenda.com/event/logique-des-passio...,Ce spectacle invite le public à vivre ou reviv...,"13, rue Pierre Sémard. 94400 Vitry-sur-Seine",Val-de-Marne,Nouveau Gare au Théâtre,Île-de-France,2022-11-19,fr,Point,"[2.403721, 48.800061]",Port à l'Anglais,20:00 - 21:00
3,Expoprotection,"[48.832528, 2.287004]",2022-11-15,"expoprotection,santé et sécurité au travail,sé...",Expoprotection,NaN,Le salon de la prévention et de la gestion des...,http://openagenda.com/event/expoprotection_640,"Tous les deux ans à Paris, Expoprotection rass...","Place de la Porte de Versailles, 75015 Paris",Paris,Porte de Versailles,Île-de-France,2022-11-17,fr,Point,"[2.287004, 48.832528]",NaN,09:00 - 18:00
4,EquipHotel,"[48.832488, 2.287499]",2022-11-06,"hôtellerie,restauration,collectivités",EquipHotel,NaN,Hotel &amp; Restaurant Business Place,http://openagenda.com/event/equiphotel_359,"Du 6 au 10 novembre 2022, ne manquez pas le re...",1 place de la Porte de Versailles,Paris,Paris expo Porte de Versailles,Île-de-France,2022-11-10,fr,Point,"[2.287499, 48.832488]",15e Arrondissement,09:30 - 19:30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Sirba Octet &amp; Balalaïka,"[48.765358, 2.268554]",2022-06-07,NaN,Sirba Octet &amp; Balalaïka,15.0,SIRBALALAÏKA,http://openagenda.com/event/test-sirba-octet-a...,"Voir un concert du Sirba Octet, c’est comme pa...","254, avenue de la Division Leclerc 92290 châte...",Hauts-de-Seine,L'Azimut - Théâtre La Piscine,Île-de-France,2022-06-07,fr,Point,"[2.268554, 48.765358]",NaN,20:30 - 22:00
96,L'Aparté vous accueille ce samedi,"[48.824529, 2.274148]",2022-06-04,"CLAVIM,Aparté,Maison Verte",L'Aparté vous accueille ce samedi,NaN,Lieu d’accueil pour les bébés et les enfants d...,http://openagenda.com/event/laparte-vous-accue...,L’**Aparté** vous accueille sans rendez-vous c...,"60 rue du Général Leclerc, 92130 Issy-les-Moul...",Hauts-de-Seine,L'Aparté,Île-de-France,2022-06-04,fr,Point,"[2.274148, 48.824529]",NaN,15:00 - 18:30
97,Erstkommunion,"[48.870856, 2.285033]",2022-06-04,NaN,Erstkommunion,NaN,Festgottesdienst der Erstkommunion in St. Hono...,http://openagenda.com/event/erstkommunion-4029827,NaN,"66 bis avenue Raymond-Poincaré, 75116 Paris",Paris,St Honoré d'Eylau,Île-de-France,2022-06-04,fr,Point,"[2.285033, 48.870856]",Paris 16e Arrondissement,10:30 - 11:50
98,Solennité de Pentecôte,"[48.855518, 2.354704]",2022-06-04,NaN,Solennité de Pentecôte,NaN,Célébrer le don de l'Esprit Saint avec les Fra...,http://openagenda.com/event/solennite-de-pente...,* **Samedi 4 juin 2022**\n\n12h30 : messe du...,"13, rue des Barres, 75004 Paris",Paris,"Église Saint-Gervais,Paris",Île-de-France,2022-06-05,fr,Point,"[2.354704, 48.855518]",Paris 4e Arrondissement,12:30 - 13:00
